# 指南

在本课程中，您将学习两个提示原则及其相关策略，以便为大型语言模型编写有效的提示。

[课程网站](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/2/guidelines)


## 准备工作

加载 API 密钥和相关的 Python 库。

在本课程中，我们提供了一些代码，帮助您加载 OpenAI API 密钥。

In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

### 辅助函数

在整个课程中，我们将使用 OpenAI 的 gpt-3.5-turbo 模型和聊天补全端点。

这个辅助函数将使得使用提示和查看生成的输出更加容易：

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## 提示原则

-   **原则1：编写清晰明确的指令**
-   **原则2：给模型足够的“思考”时间**



### 原则1：编写清晰明确的指令

#### 技巧1：使用分隔符清晰地表示输入的不同部分

-   分隔符可以是任何类似的符号：```, """, < >, `<tag> </tag>`, `:`

In [12]:
text = f"""
您应该通过表达您希望模型执行的操作提供清晰且尽可能具体。
您应该通过提供尽可能清晰和具体的说明来表达您希望模型执行的操作。
这将引导模型朝着期望的输出方向发展，并减少收到不相关或不正确响应的机会。 
不要将清晰的提示与简短的提示混淆。
在许多情况下，更长的提示可以为模型提供更多的清晰度和上下文，这可以导致更详细和相关的输出。
"""
prompt = f"""
将由三个反引号分隔的文本总结成一个句子。
```{text}```
"""
response = get_completion(prompt)
print(response)

提供清晰且尽可能具体的操作说明，以引导模型朝着期望的输出方向发展，避免混淆简短提示和长提示的作用。


#### 技巧2：要求结构化输出

-   JSON，HTML

In [5]:
prompt = f"""
生成三个虚构的中文书名及其作者和类型的清单。 
使用以下键以 JSON 格式提供它们：book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

[
  {
    "book_id": 1,
    "title": "荒野之心",
    "author": "张三",
    "genre": "冒险"
  },
  {
    "book_id": 2,
    "title": "红尘梦",
    "author": "李四",
    "genre": "言情"
  },
  {
    "book_id": 3,
    "title": "天涯孤客",
    "author": "王五",
    "genre": "历史"
  }
]


#### 技巧3：让模型检查条件是否满足

In [14]:
text_1 = f"""
泡一杯茶很简单！ 首先，你需要把水烧开。 当发生这种情况时，拿起一个杯子并在其中放入一个茶包。 一旦水足够热，就把它倒在茶包上。
让它静置一会儿，这样茶就可以陡峭了。 几分钟后，取出茶包。 如果你喜欢，你可以加一些糖或牛奶来调味。 
就是这样！ 您已经为自己准备了一杯美味的茶来享用。
"""
prompt = f"""
您将获得由三重引号分隔的文本。 如果它包含一系列指令，
按照以下格式重写这些指令：

第 1 步 - ...
第 2 步 - …
……
第 N 步 - …

如果文本不包含指令序列，
然后简单地写下“没有提供步骤。”

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
第 1 步 - 把水烧开。
第 2 步 - 拿起一个杯子并在其中放入一个茶包。
第 3 步 - 一旦水足够热，就把它倒在茶包上。
第 4 步 - 让它静置一会儿，这样茶就可以陡峭了。
第 5 步 - 几分钟后，取出茶包。
第 6 步 - 如果你喜欢，你可以加一些糖或牛奶来调味。
第 7 步 - 就是这样！ 您已经为自己准备了一杯美味的茶来享用。


In [4]:
text_2 = f"""
今天阳光灿烂，鸟儿在歌唱。 这是去公园散步的好天气。 鲜花盛开，树木在微风中轻轻摇曳。 人们出门在外，享受着宜人的天气。 
有的在野餐，有的在玩游戏，有的在草地上放松。 这是在户外度过时光并欣赏大自然之美的完美日子。
"""
prompt = f"""
您将获得由三重引号分隔的文本。 如果它包含一系列指令，
按照以下格式重写这些指令：

第 1 步 - ...
第 2 步 - …
……
第 N 步 - …

如果文本不包含指令序列，
然后简单地写下“没有提供步骤。”

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
没有提供步骤。


#### 技巧4：少示例（"Few-shot"）提示

In [15]:
prompt = f"""
你的任务是以一致的风格回答。

<孩子>：教我耐心。

<祖父母>： 雕刻最深山谷的河流来自一个谦虚的泉水； 最伟大的交响乐源于一个音符； 最复杂的挂毯始于一根单独的线。

<孩子>：教我韧性。
"""
response = get_completion(prompt)
print(response)

<祖父母>：像一棵树一样，你需要在风雨中弯曲，但不会折断。你需要学会适应变化，坚持不懈地追求你的目标。记住，韧性是一种品质，需要不断地锻炼和培养。


### 原则2：给模型足够的“思考”时间


#### 技巧1：明确完成任务所需的步骤

In [16]:
text = f"""
在一个迷人的村庄里，杰克和吉尔兄妹踏上了去山顶取水的旅程出色地。 
当他们欢呼雀跃地向上爬时，不幸降临了——杰克被一块石头绊倒，从山上滚下，吉尔也跟着跌倒。
虽然受到了轻微的打击，但两人回到家后得到了安慰的拥抱。 尽管发生了事故，他们的冒险精神丝毫未减，继续愉快地探索。
"""
# example 1
prompt_1 = f"""
执行以下操作：
1 - 总结以下文本，用 1 个句子用三重反引号分隔。
2 - 将摘要翻译成法语。
3 - 在法语摘要中列出每个名字。
4 - 输出包含以下键的 json 对象：french_summary、num_names。

用换行符分隔你的答案。

文本:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
```
杰克和吉尔兄妹在迷人的村庄里踏上了去山顶取水的旅程，但不幸中途遭遇事故，幸好两人只受轻伤，回到家后得到了安慰的拥抱，继续愉快地探索。
```
Dans un charmant village, les frères et sœurs Jack et Jill ont entrepris un voyage réussi pour aller chercher de l'eau au sommet de la montagne. Malheureusement, un accident est survenu en cours de route, mais heureusement, les deux n'ont subi que des blessures légères et ont été réconfortés par des câlins à leur retour à la maison, continuant leur exploration joyeuse. 

Noms: Jack, Jill.

{
  "french_summary": "Dans un charmant village, les frères et sœurs Jack et Jill ont entrepris un voyage réussi pour aller chercher de l'eau au sommet de la montagne. Malheureusement, un accident est survenu en cours de route, mais heureusement, les deux n'ont subi que des blessures légères et ont été réconfortés par des câlins à leur retour à la maison, continuant leur exploration joyeuse.",
  "num_names": 2
}


要求以指定格式输出

In [17]:
prompt_2 = f"""
您的任务是执行以下操作：
1 - 用 1 句话总结以下由 <> 分隔的文本。
2 - 将摘要翻译成法语。
3 - 在法语摘要中列出每个名字。
4 - 输出包含以下键的 json 对象：french_summary、num_names。

使用以下格式：
文本：<要总结的文本>
摘要：<摘要>
翻译：<摘要翻译>
姓名：<意大利语摘要中的姓名列表>
输出 JSON：<带有摘要和 num_names 的 json>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
摘要：杰克和吉尔兄妹在旅途中遭遇不幸，但最终平安回家，继续探险。
翻译：Jack et Jill, frère et sœur, ont eu un accident lors de leur voyage pour chercher de l'eau au sommet de la montagne, mais ils sont rentrés chez eux en sécurité et ont continué leur exploration avec leur esprit d'aventure intact.
姓名：杰克、吉尔
输出 JSON：{"french_summary": "Jack et Jill, frère et sœur, ont eu un accident lors de leur voyage pour chercher de l'eau au sommet de la montagne, mais ils sont rentrés chez eux en sécurité et ont continué leur exploration avec leur esprit d'aventure intact.", "num_names": 2}


#### 技巧2：指导模型在得出结论之前先自己解决问题

In [18]:
prompt = f"""
判断学生的答案是否正确。

问题：
我正在建造一个太阳能装置，我需要帮助来计算财务费用。
- 土地成本 100 美元/平方英尺
- 我可以以 250 美元/平方英尺的价格购买太阳能电池板
- 我协商了一份维护合同，每年将花费我 10 万美元，外加每平方英尺 10 美元

运营第一年的总成本与平方英尺数的关系是多少？

学生的解决方案：
设 x 是以平方英尺为单位的安装尺寸。
费用：
1.土地成本：100x
2.太阳能电池板成本：250x
3.维护费用：100,000+100x
总成本：100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

学生的答案是正确的。


请注意，学生的解决方案实际上是错误的。 我们可以通过指导模型先自己解决问题来解决这个问题。

In [19]:
prompt = f"""
您的任务是确定学生的解决方案是否正确。
要解决此问题，请执行以下操作：
- 首先，找出你自己的解决方案。
- 然后将您的解决方案与学生的解决方案进行比较，并评估学生的解决方案是否正确。

在您自己完成问题之前，不要判断学生的解决方案是否正确。

使用以下格式：
问题：
```
在这里提问
```
学生的解决方案：
```
学生的解决方案在这里
```
实际解决方案：
```
在此处制定解决方案和您的解决方案的步骤
```
学生的解法和实际解法一样吗 \
刚刚计算：
```
是还是不是
```
学生年级：
```
正确或不正确
```

问题：
```
我正在建造一个太阳能装置，我需要帮助来计算财务费用。
- 土地成本 100 美元/平方英尺
- 我可以以 250 美元/平方英尺的价格购买太阳能电池板
- 我协商了一份维护合同，每年将花费我 10 万美元，外加每平方英尺 10 美元

运营第一年的总成本与平方英尺数的关系是多少？
```
学生的解决方案：
```
设 x 是以平方英尺为单位的安装尺寸。
费用：
1.土地成本：100x
2.太阳能电池板成本：250x
3.维护费用：100,000+100x
总成本：100x + 250x + 100,000 + 100x = 450x + 100,000
```
实际解决方案：
"""
response = get_completion(prompt)
print(response)

学生的解决方案是正确的。

运营第一年的总成本与平方英尺数的关系是：总成本 = 450x + 100,000，其中 x 是以平方英尺为单位的安装尺寸。


## 模型限制：幻觉

- Boie 是一家真实的公司，产品名称并非真实存在。

In [20]:
prompt = f"""
告诉我有关 Boie 的 AeroGlide UltraSlim 智能牙刷的信息
"""
response = get_completion(prompt)
print(response)

Boie的AeroGlide UltraSlim智能牙刷是一款高科技的电动牙刷，采用了先进的声波技术，能够提供高效的清洁效果。该牙刷的刷头非常细小，能够轻松进入牙缝和牙龈，清洁效果非常出色。此外，该牙刷还配备了智能感应技术，能够自动调节清洁力度和时间，让你的牙齿更加健康。该牙刷还具有防水设计，可以在淋浴或洗脸时使用，非常方便。总之，Boie的AeroGlide UltraSlim智能牙刷是一款高品质、高效率的电动牙刷，非常适合追求健康口腔的人士使用。


### 关于在本课程外使用 OpenAI API 的说明

要安装 OpenAI Python 库：

```
!pip install openai
```

您需要使用账户密钥配置库，密钥可在[网站](https://platform.openai.com/account/api-keys)上找到。

您可以在使用库之前将其设置为 `OPENAI_API_KEY` 环境变量：

 ```
 !export OPENAI_API_KEY='sk-...'
 ```

或者，将 `openai.api_key` 设置为其值：

```
import openai
openai.api_key = "sk-..."
```


### 关于反斜杠的说明

-   在本课程中，我们使用反斜杠 `\` 来使文本适应屏幕，而不需要插入换行符 '\n'。
-   GPT-3 对于插入换行符与否并不受太大影响。但是在一般使用大型语言模型（LLMs）时，您可能需要考虑在提示中的换行符是否会影响模型的性能。